In [1]:
import re
import csv
import time
import nltk
import numpy as np
from gensim import utils
from gensim.models import KeyedVectors
import gensim.models
from tqdm import tqdm
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')

csv.field_size_limit(100000000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


131072

In [2]:
class TCUCorpus(object):
    """An interator that yields sentences (lists of str)."""
    
    @staticmethod
    def clean_text(text):
        #source: https://medium.com/ml2vec/using-word2vec-to-analyze-reddit-comments-28945d8cee57
        
        #Normalize tabs and remove newlines
        no_tabs = str(text).replace('\t', ' ').replace('\n', '');
        
        #Remove all characters except A-Z
        alphas_only = re.sub("[^a-zA-Z]", " ", no_tabs);
        
        #Normalize spaces to 1
        multi_spaces = re.sub(" +", " ", alphas_only);
        
        #Strip trailing and leading spaces
        no_spaces = multi_spaces.strip();
        
        #Remove stopwords 
        stopwords = nltk.corpus.stopwords.words('portuguese')
        clean_text = [w for w in no_spaces.split() if not w in stopwords] 

        return ' '.join(clean_text)
    
    @staticmethod
    def preprocess(text):
        return utils.simple_preprocess(TCUCorpus.clean_text(text))
                

    def __iter__(self):
        filename = 'datasets/acordaos_relator_5k.csv'
        with open(filename, "r") as csvfile:
            corpus = csv.reader(csvfile)
            next(corpus) #ignores header
            for doc in tqdm(corpus):
                yield TCUCorpus.preprocess(doc[1])

In [3]:
def train_w2v(corpus, sg=0):
#     sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
# In the CBOW model, the distributed representations of context (or surrounding words) 
# are combined to predict the word in the middle. 
# While in the Skip-gram model, the distributed representation of the input word is used 
# to predict the context.
# https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314
    model = gensim.models.Word2Vec(sentences=corpus, min_count=50, size=100, workers=4, sg=sg)
    return model

def load_w2v(path='model.bin'):
    model = gensim.models.Word2Vec.load(path)
    return model

def load_pretrained_w2v(path):
    model = KeyedVectors.load_word2vec_format(path)
    return model

In [4]:
# source: https://github.com/v1shwa/document-similarity/blob/master/DocSim.py
class DocW2V:
    
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
        
    def transform(self, docs):
        return np.array([self.vectorize(doc) for doc in docs])

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        
        words = TCUCorpus.preprocess(doc)
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model.wv[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

In [5]:
docs = pd.read_csv('datasets/acordaos_relator_5k.csv')
# corpus = TCUCorpus()
# model = train_w2v(corpus)
# model.save('model.bin')
model_trained = load_w2v('model.bin')
# http://www.nilc.icmc.usp.br/embeddings - CBOW 100
model_pretrained = load_pretrained_w2v('cbow_s100.txt')

In [6]:
tfidf = TfidfVectorizer()
tfidf.fit([TCUCorpus.clean_text(doc) for doc in docs])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [7]:
X = np.array(docs.acordao)
le = preprocessing.LabelEncoder()
y = le.fit_transform(np.array(docs.relator.tolist()))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

embedders = [
    tfidf, 
    DocW2V(model_pretrained), 
    DocW2V(model_trained)
]
embedders_map = { 0:'TFIDF', 1:'PRETRAINED', 2:'TRAINED'}
data_embedded = []
for embedder in tqdm(embedders):
    X_train_enc = embedder.transform(X_train)
    X_test_enc = embedder.transform(X_test)
    data_embedded.append((X_train_enc, X_test_enc))

100%|██████████| 3/3 [15:36<00:00, 312.18s/it]


In [9]:
results = []
clfs = [KNeighborsClassifier(n_neighbors=7),
        SVC(gamma='auto', random_state=42), 
        MLPClassifier(random_state=42, max_iter=300)]
clfs_map = { 0:'KNN', 1:'SVM', 2:'MLP'}
kf = KFold(n_splits=10)
for train_index, test_index in tqdm(kf.split(X_train)):
    for ci, clf in enumerate(clfs):
        for ei, data in enumerate(data_embedded):
            X_enc_train, X_enc_test = data
            X_, X_validation = X_enc_train[train_index], X_enc_train[test_index]
            y_, y_validation = y_train[train_index], y_train[test_index]
            startTime = time.perf_counter()
            
            clf.fit(X_, y_)
            y_pred = clf.predict(X_validation)
            
            elapsed_time = time.perf_counter() - startTime
            
            p, r, f, s = precision_recall_fscore_support(y_validation, y_pred, average='micro')
            
            result = {"CLASSIFIER": clfs_map[ci],
                     "EMBEDDER": embedders_map[ei],
                     "PRECISION": p,
                     "RECALL":r,
                     "F1SCORE":f,
                     "TIME": elapsed_time}
            print(result)
            results.append(result)
df = pd.DataFrame(results) 
df.to_csv('results.csv')

0it [00:00, ?it/s]

{'CLASSIFIER': 'KNN', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.11129185305679816, 'RECALL': 0.11129185305679816, 'F1SCORE': 0.11129185305679816, 'TIME': 71.49899262399776}
{'CLASSIFIER': 'KNN', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.714111427409516, 'RECALL': 0.714111427409516, 'F1SCORE': 0.714111427409516, 'TIME': 507.5953508349994}
{'CLASSIFIER': 'KNN', 'EMBEDDER': 'TRAINED', 'PRECISION': 0.7246396457457864, 'RECALL': 0.7246396457457864, 'F1SCORE': 0.7246396457457864, 'TIME': 192.03062428599878}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.1661470335728164, 'RECALL': 0.1661470335728164, 'F1SCORE': 0.1661470335728164, 'TIME': 1814.2494784329974}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.34697935023270526, 'RECALL': 0.34697935023270526, 'F1SCORE': 0.34697935023270526, 'TIME': 10349.277052106001}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'TRAINED', 'PRECISION': 0.7308300573855677, 'RECALL': 0.7308300573855677, 'F1SCORE': 0.7308300573855677, 'TIME': 4906.7708052480

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'CLASSIFIER': 'MLP', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.704667660747368, 'RECALL': 0.704667660747368, 'F1SCORE': 0.704667660747368, 'TIME': 1746.1100166589968}


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
1it [5:32:21, 19941.80s/it]

...

{'CLASSIFIER': 'MLP', 'EMBEDDER': 'TRAINED', 'PRECISION': 0.7660295513081198, 'RECALL': 0.7660295513081198, 'F1SCORE': 0.7660295513081199, 'TIME': 333.33021656799247}
{'CLASSIFIER': 'KNN', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.16456554154805478, 'RECALL': 0.16456554154805478, 'F1SCORE': 0.16456554154805478, 'TIME': 69.70296594599495}
{'CLASSIFIER': 'KNN', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.7212959197505762, 'RECALL': 0.7212959197505762, 'F1SCORE': 0.7212959197505762, 'TIME': 503.90887529900647}
{'CLASSIFIER': 'KNN', 'EMBEDDER': 'TRAINED', 'PRECISION': 0.7270344765261398, 'RECALL': 0.7270344765261398, 'F1SCORE': 0.72703447652614, 'TIME': 188.77443715601112}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.16461072703447652, 'RECALL': 0.16461072703447652, 'F1SCORE': 0.16461072703447652, 'TIME': 1725.4063511080167}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.34982603587727623, 'RECALL': 0.34982603587727623, 'F1SCORE': 0.34982603587727623, 'TIME': 10121.5923

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'CLASSIFIER': 'MLP', 'EMBEDDER': 'PRETRAINED', 'PRECISION': 0.705978039853599, 'RECALL': 0.705978039853599, 'F1SCORE': 0.705978039853599, 'TIME': 2001.261262097978}


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
10it [55:51:41, 20110.15s/it]

{'CLASSIFIER': 'MLP', 'EMBEDDER': 'TRAINED', 'PRECISION': 0.7671140029822421, 'RECALL': 0.7671140029822421, 'F1SCORE': 0.7671140029822421, 'TIME': 329.1742454560008}


* A escolha pela representação CBOW se dá pela grande quantidade de dados usados no experimento, pois usar o skip-gram afetaria muito o tempo de treinamento.
* O experimento que gerou os resultados foi completado em 55 horas, 51 minutos e 41 segundos.
* A escolha da dimensão dos _word vectors_ foi devido a limitações do hardware do experimento.

In [14]:
tfidf_ = TfidfVectorizer()
tfidf_.fit([TCUCorpus.clean_text(doc) for doc in docs.acordao])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [25]:
new_df = df[df.EMBEDDER != 'TFIDF']
new_df.reset_index(drop=True)

,CLASSIFIER,EMBEDDER,PRECISION,RECALL,F1SCORE,TIME
0,KNN,PRETRAINED,0.714111,0.714111,0.714111,507.595351
1,KNN,TRAINED,0.724640,0.724640,0.724640,192.030624
2,SVM,PRETRAINED,0.346979,0.346979,0.346979,10349.277052
3,SVM,TRAINED,0.730830,0.730830,0.730830,4906.770805
4,MLP,PRETRAINED,0.704668,0.704668,0.704668,1746.110017
5,MLP,TRAINED,0.768470,0.768470,0.768470,337.229806
6,KNN,PRETRAINED,0.714925,0.714925,0.714925,507.439416
7,KNN,TRAINED,0.722335,0.722335,0.722335,191.162868
8,SVM,PRETRAINED,0.345217,0.345217,0.345217,10338.410488
9,SVM,TRAINED,0.730966,0.730966,0.730966,4790.256861


In [26]:
for embedder in tqdm(embedders):
    new_X_train_enc = tfidf_.transform(X_train)
    new_X_test_enc = tfidf_.transform(X_test)

100%|██████████| 3/3 [02:40<00:00, 53.54s/it]


In [ ]:
new_results = []

for train_index, test_index in tqdm(kf.split(X_train)):
    for ci, clf in enumerate(clfs):
        X_, X_validation = new_X_train_enc[train_index], new_X_test_enc[test_index]
        y_, y_validation = y_train[train_index], y_train[test_index]
        startTime = time.perf_counter()

        clf.fit(X_, y_)
        y_pred = clf.predict(X_validation)

        elapsed_time = time.perf_counter() - startTime

        p, r, f, s = precision_recall_fscore_support(y_validation, y_pred, average='micro')

        result = {"CLASSIFIER": clfs_map[ci],
                 "EMBEDDER": 'TFIDF',
                 "PRECISION": p,
                 "RECALL":r,
                 "F1SCORE":f,
                 "TIME": elapsed_time}
        print(result)
        new_results.append(result)

0it [00:00, ?it/s]

{'CLASSIFIER': 'KNN', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.06506710044733631, 'RECALL': 0.06506710044733631, 'F1SCORE': 0.06506710044733631, 'TIME': 640.0019952070434}
{'CLASSIFIER': 'SVM', 'EMBEDDER': 'TFIDF', 'PRECISION': 0.11124666757037639, 'RECALL': 0.11124666757037639, 'F1SCORE': 0.11124666757037639, 'TIME': 55685.40869214601}
